In [242]:
import pandas as pd
pd.set_option('max_columns',1000)
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import decomposition
from sklearn.metrics import average_precision_score
import pickle
from sklearn.metrics import mean_absolute_error as MAE
import datetime

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
import xgboost as xgb
from xgboost import XGBClassifier
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn import preprocessing
from scipy import stats
from sklearn.linear_model import LogisticRegression

In [2]:
# Read data from local directory, remember to change the directory
listing = pd.read_csv('roofstock_marketplace_listing_historical_full.csv')
transaction = pd.read_csv('roofstock_marketplace_transactions_full.csv')
offer = pd.read_csv('roofstock_marketplace_offers_full.csv')
buyer = pd.read_csv('roofstock_marketplace_buyers_full.csv')

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11,14,38,42,43,57,58,59,64,65,66,80,81,84,132,141,151,166,167,168,169,170) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [243]:

newdf=pd.read_csv('without_estimate.csv')
newdf['DISCOUNT']=newdf['OFFER_PRICE']/newdf['LIST_PRICE']
zesti=pd.read_csv('zestimate_data.csv')
zesti_dic=zesti.set_index('LISTING_ID')['zestimate'].to_dict()
newdf['ZESTIMATE']=newdf['LISTING_ID'].map(lambda x: zesti_dic[x])
#newdf['ZESTIMATE_DISCOUNT']=newdf['LIST_PRICE']/newdf['ZESTIMATE']
newdf['ZESTIMATE_DIFFERENCE']=newdf['LIST_PRICE']-newdf['ZESTIMATE']

newdf['OFFER_OR_NOT']=newdf['diff_days'].map(lambda x: 1 if x<180 else 0)
newdf['OFFER_OR_NOT']=newdf['OFFER_OR_NOT'].astype('category')
INSPECTION_TYPE_ID=pd.get_dummies(newdf['INSPECTION_TYPE_ID'],dummy_na=True,prefix='INSPECTION_TYPE_ID')
newdf=pd.concat([newdf,INSPECTION_TYPE_ID],axis=1)
newdf=newdf.drop('INSPECTION_TYPE_ID',axis=1)

ddrop=[]
for i in newdf.columns:
    if 'LEASING' in i:
        ddrop.append(i)

for i in newdf.columns:
    if 'PAYMENTSTATUS' in i:
        ddrop.append(i)

newdf=newdf.drop(ddrop,axis=1)
newdf=newdf.drop(['GMAPPOVHEADING','GMAPPOVPITCH','ISALLOWOFFER','PRICEVISIBILITY','HASAUDIO','ISEXCLUSIVE'],axis=1)

In [4]:
df_has_offer=newdf[newdf['diff_days']<=180]
df_has_offer=df_has_offer[df_has_offer['LISTING_STATUS_For Sale']==1]
ddrop1=[]
for i in newdf.columns:
    if 'LISTING_STATUS' in i:
        ddrop1.append(i)
df_has_offer=df_has_offer.drop(ddrop1[1:],axis=1)

In [5]:
df_has_offer=df_has_offer.groupby('LISTING_ID').apply(lambda t: t[t.EVENT_UTC==t.EVENT_UTC.min()]).drop_duplicates()

In [6]:
df_has_offer=df_has_offer.drop_duplicates(subset=['LISTING_ID','EVENT_UTC'],keep='first')

In [7]:
len(df_has_offer)

1255

In [8]:
df_has_offer=df_has_offer.drop('LISTING_ID',axis=1).reset_index().drop('level_1',axis=1)

In [9]:
df_has_offer.head()

,LISTING_ID,LISTING_STATUS_IS_ACTIVE,LOTSIZE,SQUAREFEET,BEDROOMS,BATHROOMS,ISPOOL,ISHOA,GMAPHASSTREETVIEW,SCHOOLELEMENTARYSCORE,SCHOOLMIDDLESCORE,SCHOOLHIGHSCORE,ISALLOWPREVIEW,ISFEATURED,ISRENTGUARANTEED,ISSECURITIZED,INSPECTION_CONTINGENCY_REQUIRED,ISELIGIBLEFORADS,NEIGHBORHOODSCORE,LIST_PRICE,MONTHLY_RENT,MARKET_RENT,MONTHLYHOA,YEARLYPROPERTYTAXES,YEARLYINSURANCECOST,PREVIOUSYEARLYPROPERTYTAXES,EST_REPAIR_COST,ASSUMED_DOWNPAYMENT_TYPE,COMPUTED_GROSS_YIELD,COMPUTED_CAP_RATE,COMPUTEDTOTALEXPENSES,COMPUTEDTOTALEXPENSESPERCENTAGE,COMPUTEDCASHFLOW,COMPUTEDAPPRECIATION,COMPUTEDLEVEREDIRR,COMPUTEDLEVEREDCASHONCASH,COMPUTEDLEVEREDNETYIELD,COMPUTEDLEVEREDANNUALCASHFLOW,COMPUTEDTOTALRETURN,COMPUTEDTOTALOPERATINGEXPENSEPERCENT,COMPUTEDLEVEREDINITIALOUTLAY,COMPUTEDLOANPAYMENTS,COMPUTEDEXPENSESANDRESERVES,COMPUTEDEXPECTEDRENT,COMPUTEDDOWNPAYMENTPERCENTAGE,COMPUTEDPROPERTYTAXES,OCCUPANCY,LEASEMONTHLYRENT,SECURITYDEPOSITAMOUNT,HASPET,ISPETSDEPOSIT,ISLEASECONCESSIONS,ISSECTION8,ISRENTERSINSURANCEREQUIRED,ISTENANTBACKGROUNDCHECKED,ISTENANTINCOMEABOVE3X,ISTENANTMAYTERMINATEEARLY,ISTENANTPURCHASEOPTION,OWNERELECTRIC,OWNERGAS,OWNERWATER,OWNERGARBAGE,OWNERPOOL,OWNERLANDSCAPING,OWNERPESTCONTROL,OWNERREFRIGERATOR,OWNERDISHWASHER,OWNERWASHER,OWNERDRYER,OWNERMICROWAVE,OWNERSTOVE,OFFER_PRICE,diff_days,HOUSEAGE,EVENT_UTC,LISTING_INITIAL_PUBLISH_TS,ALLOWABLE_SALE_TYPES_1,ALLOWABLE_SALE_TYPES_3,ALLOWABLE_SALE_TYPES_5,ALLOWABLE_SALE_TYPES_7,ALLOWABLE_SALE_TYPES_9,ALLOWABLE_SALE_TYPES_33,ALLOWEDFUNDINGTYPES_-9999.0,ALLOWEDFUNDINGTYPES_1.0,ALLOWEDFUNDINGTYPES_4.0,ALLOWEDFUNDINGTYPES_5.0,ALLOWEDFUNDINGTYPES_16.0,ALLOWEDFUNDINGTYPES_17.0,ALLOWEDFUNDINGTYPES_21.0,CBSA_CODE_-99999.0,CBSA_CODE_10420.0,CBSA_CODE_11500.0,CBSA_CODE_12020.0,CBSA_CODE_12060.0,CBSA_CODE_12100.0,CBSA_CODE_12260.0,CBSA_CODE_12420.0,CBSA_CODE_12580.0,CBSA_CODE_13140.0,CBSA_CODE_13820.0,CBSA_CODE_15940.0,CBSA_CODE_15980.0,CBSA_CODE_16340.0,CBSA_CODE_16740.0,CBSA_CODE_16980.0,CBSA_CODE_17140.0,CBSA_CODE_17260.0,CBSA_CODE_17300.0,CBSA_CODE_17460.0,CBSA_CODE_17900.0,CBSA_CODE_17980.0,CBSA_CODE_18020.0,CBSA_CODE_18140.0,CBSA_CODE_19100.0,CBSA_CODE_19380.0,CBSA_CODE_19460.0,CBSA_CODE_19500.0,CBSA_CODE_19660.0,CBSA_CODE_19820.0,CBSA_CODE_20500.0,CBSA_CODE_22180.0,CBSA_CODE_22220.0,CBSA_CODE_23060.0,CBSA_CODE_23580.0,CBSA_CODE_24580.0,CBSA_CODE_24660.0,CBSA_CODE_24860.0,CBSA_CODE_25420.0,CBSA_CODE_26420.0,CBSA_CODE_26540.0,CBSA_CODE_26620.0,CBSA_CODE_26900.0,CBSA_CODE_27140.0,CBSA_CODE_27260.0,CBSA_CODE_27340.0,CBSA_CODE_27600.0,CBSA_CODE_28140.0,CBSA_CODE_28660.0,CBSA_CODE_29460.0,CBSA_CODE_29820.0,CBSA_CODE_31080.0,CBSA_CODE_31140.0,CBSA_CODE_31260.0,CBSA_CODE_31420.0,CBSA_CODE_32820.0,CBSA_CODE_32940.0,CBSA_CODE_33100.0,CBSA_CODE_33140.0,CBSA_CODE_33340.0,CBSA_CODE_33460.0,CBSA_CODE_33860.0,CBSA_CODE_34620.0,CBSA_CODE_34980.0,CBSA_CODE_35620.0,CBSA_CODE_35840.0,CBSA_CODE_36420.0,CBSA_CODE_36700.0,CBSA_CODE_36740.0,CBSA_CODE_37340.0,CBSA_CODE_37980.0,CBSA_CODE_38060.0,CBSA_CODE_38240.0,CBSA_CODE_38300.0,CBSA_CODE_38940.0,CBSA_CODE_39460.0,CBSA_CODE_39580.0,CBSA_CODE_40140.0,CBSA_CODE_40420.0,CBSA_CODE_40660.0,CBSA_CODE_41060.0,CBSA_CODE_41180.0,CBSA_CODE_41700.0,CBSA_CODE_41860.0,CBSA_CODE_42340.0,CBSA_CODE_42860.0,CBSA_CODE_43780.0,CBSA_CODE_44180.0,CBSA_CODE_44940.0,CBSA_CODE_45300.0,CBSA_CODE_45780.0,CBSA_CODE_45940.0,CBSA_CODE_46220.0,CBSA_CODE_46700.0,CBSA_CODE_47580.0,CBSA_CODE_48900.0,CBSA_CODE_49180.0,CBSA_CODE_49620.0,CERTIFICATIONLEVEL_-9999.0,CERTIFICATIONLEVEL_0.0,CERTIFICATIONLEVEL_50.0,CERTIFICATIONLEVEL_100.0,TITLESTATUS_0.0,TITLESTATUS_1.0,TITLESTATUS_2.0,TITLESTATUS_nan,FLOODRISKSCORE_A,FLOODRISKSCORE_AE,FLOODRISKSCORE_AH,FLOODRISKSCORE_D,FLOODRISKSCORE_Null,FLOODRISKSCORE_X,FLOODRISKSCORE_Y,FLOODRISKSCORE_nan,VISIBILITY_ID_0,VISIBILITY_ID_10,VISIBILITY_ID_20,KITCHENCONDITIONID_-9999.0,KITCHENCONDITIONID_110.0,KITCHENCONDITIONID_120.0,KITCHENCONDITIONID_210.0,KITCHENCONDITIONID_310.0,KITCHENCONDITIONID_320.0,KITCHENCONDITIONID_410.0,LANDSCAPINGCONDITIONID_-9999.0,LANDSCAPING

In [69]:
# Survival analysis
listing['REC_END_TS']=listing['REC_END_TS'].map(lambda x: x[:10])
listing=listing[listing['LIST_PRICE'].isnull()==False]
df_list=[]
for i in tqdm(df_has_offer.index):
    try:
        offer_date=df_has_offer['EVENT_UTC'][i][:10]
        initial_publish_date=df_has_offer['LISTING_INITIAL_PUBLISH_TS'][i][:10]
        listing_id=df_has_offer['LISTING_ID'][i]
        listing1=listing[(listing['LISTING_ID']==listing_id)&(listing['REC_END_TS']<=offer_date)&(listing['REC_END_TS']>=initial_publish_date)]
        listing1['REC_END_TS']=listing1['REC_END_TS'].map(lambda x:x[:10])
        xx=listing1[['LISTING_ID','REC_END_TS','LISTING_INITIAL_PUBLISH_TS','LIST_PRICE']].sort_values('REC_END_TS').drop_duplicates(subset=['REC_END_TS'],keep='first')
        xx1=xx.reset_index()
        initial_list_price=xx1['LIST_PRICE'][0]
        xx=xx.dropna(subset=['LISTING_INITIAL_PUBLISH_TS'])
        xx['LISTING_INITIAL_PUBLISH_TS']=xx['LISTING_INITIAL_PUBLISH_TS'].map(lambda x:x[:10])
        xx['REC_END_TS']=pd.to_datetime(xx['REC_END_TS'])
        xx['LISTING_INITIAL_PUBLISH_TS']=pd.to_datetime(xx['LISTING_INITIAL_PUBLISH_TS'])
        xx['DAYS_ON_MARKET']=xx['REC_END_TS']-xx['LISTING_INITIAL_PUBLISH_TS']
        xx['DAYS_ON_MARKET']=xx['DAYS_ON_MARKET'].map(lambda x: x.days)
        diff_days=max(2,(df_has_offer.iloc[i:i+1,]['diff_days'][i]))
        s_df=pd.concat([(df_has_offer.iloc[i:i+1,])]*int(diff_days-1))
        s_df=s_df.reset_index().drop('index',axis=1)
        s_df['DAY_ON_MARKET']=list(range(int(diff_days-1)))
        s_df['INITIAL_LIST_PRICE']=initial_list_price
        today_price_list=[]
        for x in s_df.index:
            try:
                mm=xx[xx['DAYS_ON_MARKET']<=s_df['DAY_ON_MARKET'][x]]
                today_price_list.append(mm[mm['DAYS_ON_MARKET']==mm['DAYS_ON_MARKET'].max()]['LIST_PRICE'].iloc[0,])
            except:
                today_price_list.append(initial_list_price)
        s_df['CURRENT_LIST_PRICE']=today_price_list
        s_df['OFFER_TOMORROW']=0
        s_df=s_df.set_value(s_df.index[-1],'OFFER_TOMORROW',1)
        df_list.append(s_df)
    except:
        print(i)
df_survival=pd.concat(df_list)
df_survival=df_survival.reset_index().drop('index',axis=1)

  0%|          | 0/1255 [00:00<?, ?it/s]D:\anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
D:\anaconda\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
100%|██████████| 1255/1255 [05:40<00:00,  5.48it/s]


In [195]:
dd_list=[]
for i in tqdm(df_survival['LISTING_ID'].unique()):
    aa=df_survival[df_survival['LISTING_ID']==i]
    bb=aa.tail(1)
    cc=bb.copy()
    cc['OFFER_TOMORROW']=0
    dd_list.append(pd.concat([aa]+[cc]*(179-len(aa))))

100%|██████████| 1255/1255 [06:07<00:00,  2.72it/s]


In [196]:
df_survival2=pd.concat(dd_list)

In [258]:

train_id=random.sample(list(df_survival['LISTING_ID'].unique()),int(len(list(df_survival['LISTING_ID'].unique()))*0.6))
test_id=list(set(df_survival['LISTING_ID'].unique())-set(train_id))
df_survival1=df_survival2.drop(['EVENT_UTC','OFFER_OR_NOT',
                                                                 'LISTING_INITIAL_PUBLISH_TS','DISCOUNT','OFFER_OR_NOT','ZESTIMATE',
                                                                 'PREVIOUSYEARLYPROPERTYTAXES'],axis=1)  
df_survival1['OFFER_TOMORROW']=df_survival1['OFFER_TOMORROW'].astype('int')                     
                      
X_train=df_survival1[df_survival1['LISTING_ID'].isin(train_id)].drop(['diff_days','OFFER_TOMORROW','LISTING_ID'],axis=1)
X_test=df_survival1[df_survival1['LISTING_ID'].isin(test_id)].drop(['diff_days','OFFER_TOMORROW','LISTING_ID'],axis=1) 
y_train=df_survival1[df_survival1['LISTING_ID'].isin(train_id)]['OFFER_TOMORROW']
y_test=df_survival1[df_survival1['LISTING_ID'].isin(test_id)]['OFFER_TOMORROW']

In [259]:
abcc=[]
for i in range(len(df_survival1.columns)):
    if 'COMPUTED' in df_survival1.columns[i]:
        abcc.append(i)

PCA_computed = PCA(n_components=0.99995)
PCA_com = PCA_computed.fit(df_survival1.iloc[:,27:45])
#principal_com_Df = pd.DataFrame(data = PCA_com
             #, columns = ['COM_PC1', 'COM_PC2','COM_PC3','COM_PC4','COM_PC5','COM_PC6','COM_PC7','COM_PC8'])
com_df=pd.DataFrame(PCA_computed.transform(df_survival1.iloc[:,27:45]))
com_df.columns=['COM_PC1', 'COM_PC2','COM_PC3','COM_PC4','COM_PC5','COM_PC6','COM_PC7','COM_PC8']


df_survival1=df_survival1.drop(df_survival.iloc[:,27:45], axis=1)
df_survival1=df_survival1.join(com_df)

# Random forest

In [260]:
rf=RandomForestClassifier()
rf_model=rf.fit(X_train,y_train)
rf_pred=rf.predict(X_test)
rf_acc=accuracy_score(rf_pred,y_test)
print('random forest accuracy for predicting offer_or_not is '+str(rf_acc))
#print('random forest auc: '+str(roc_auc_score(rf_pred,y_test)))
print('random forest average precision score: '+str(average_precision_score(rf_pred,y_test)))

importances = rf.feature_importances_
rf_importance_df=pd.DataFrame({'feature':list(df_survival.drop(['diff_days','EVENT_UTC','LISTING_ID','OFFER_TOMORROW','OFFER_OR_NOT',
                                                                 'LISTING_INITIAL_PUBLISH_TS','DISCOUNT','OFFER_OR_NOT','ZESTIMATE',
                                                                 'PREVIOUSYEARLYPROPERTYTAXES','OFFER_TOMORROW'],
                                                         axis=1).columns),'importance':importances})
rf_importance_df_order=rf_importance_df[rf_importance_df['importance']>0].sort_values('importance',ascending=False)
rf_importance_df_order[:20].plot.bar(x='feature')
rf_proba=rf.predict_proba(X_test)[:,1]
tt=X_test.copy()
tt['PROBA']=rf_proba
tt['LISTING_ID']=list(df_survival1[df_survival1['LISTING_ID'].isin(test_id)]['LISTING_ID'])
tt['DIFF_DAYS']=list(df_survival1[df_survival1['LISTING_ID'].isin(test_id)]['diff_days'])
tt=tt.groupby('LISTING_ID').apply(lambda t: t[t.PROBA==t.PROBA.max()]).drop_duplicates()
tt=tt.drop('LISTING_ID',axis=1).reset_index().drop('level_1',axis=1)
tt=tt.drop_duplicates('LISTING_ID',keep='last')
print('mae for survival analysis for random forest: '+str(np.mean(abs(tt['DIFF_DAYS']-tt['DAY_ON_MARKET']))))
print('RMSE for survival analysis for random forest: '+str(np.mean((tt['DIFF_DAYS'])**2-(tt['DAY_ON_MARKET'])**2)))

D:\anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


random forest accuracy for predicting offer_or_not is 0.994413407821229


D:\anaconda\lib\site-packages\sklearn\metrics\ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


random forest average precision score: nan


KeyError: "['EVENT_UTC' 'OFFER_OR_NOT' 'LISTING_INITIAL_PUBLISH_TS' 'DISCOUNT'\n 'OFFER_OR_NOT' 'ZESTIMATE' 'PREVIOUSYEARLYPROPERTYTAXES'] not found in axis"

# logistic regression

In [221]:
rf=LogisticRegression()
rf_model=rf.fit(X_train,y_train)
rf_pred=rf.predict(X_test)
rf_acc=accuracy_score(rf_pred,y_test)
print('random forest accuracy for predicting offer_or_not is '+str(rf_acc))
#print('random forest auc: '+str(roc_auc_score(rf_pred,y_test)))
print('random forest average precision score: '+str(average_precision_score(rf_pred,y_test)))


tt=X_test.copy()
tt['PROBA']=rf_proba
tt['LISTING_ID']=list(df_survival1[df_survival1['LISTING_ID'].isin(test_id)]['LISTING_ID'])
tt['DIFF_DAYS']=list(df_survival1[df_survival1['LISTING_ID'].isin(test_id)]['diff_days'])
tt=tt.groupby('LISTING_ID').apply(lambda t: t[t.PROBA==t.PROBA.max()]).drop_duplicates()
tt=tt.drop('LISTING_ID',axis=1).reset_index().drop('level_1',axis=1)
tt=tt.drop_duplicates('LISTING_ID',keep='last')
print('mae for survival analysis for random forest: '+str(np.mean(abs(tt['DIFF_DAYS']-tt['DAY_ON_MARKET']))))
print('RMSE for survival analysis for random forest: '+str((np.mean((tt['DIFF_DAYS'])**2-(tt['DAY_ON_MARKET'])**2)))**0.5

D:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\anaconda\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


random forest accuracy for predicting offer_or_not is 0.994413407821229


D:\anaconda\lib\site-packages\sklearn\metrics\ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


random forest average precision score: nan
mae for survival analysis for random forest: 3.4721115537848606
RMSE for survival analysis for random forest: 19.244132960111614
